#### Scrapper

In [ ]:
import json
import requests
import pandas as pd
import numpy as np
import time
import os

In [ ]:
class StockTwitAPIScrapper:
  def __init__(self):
    self.intro()
    self.url = "https://api.stocktwits.com/api/2/streams/symbol/{}.json?"
    self.maxId = None
    self.symbol = None
    self.lastRun = None
    self.lastStatus = None
    self.df = pd.DataFrame()
    self.getInput()

  def intro(self):
    message = 'for this scapper it scrap and output dataframe with multiple fields, and currently only support input for 1 company symbol'
    instruction = 'pls input some constrains for this scrapper'
    print(message)
    print(instruction)

  def getInput(self):
    self.maxId = input('by default scrapper start from current utc time backwards, press enter, else pls enter the max id for the data you want to scrap:')
    while len(self.maxId) != 9 or not self.maxId.isdigit():
      if self.maxId == '':
        break
      self.maxId = input('your max id is not valid, pls re-enter:')
    self.symbol = input('pls enter the company symbol that you want to scrap:')
    while requests.get(self.url.format(self.symbol)).status_code != 200:
      self.symbol = input('seems there is no such company, pls re-enter the company symbol that you want to scrap:')

  def scrap(self, continue_last_run = False, silent = False):

    if self.lastRun is not None:
      continue_last_run = True

    if self.symbol == None:
      print('you did not set the symbol, pls re-initialize a instance')
    
    #query_times = input('1 query can get 30 entries, how many queries do you want to execute:')
    query_times = '200'
    while not query_times.isdigit():
      query_times = input('pls enter valid number:')
    query_times = int(query_times)
    while abs(query_times) > 1000:
      query_times = 1000
      #print('API has a limit 1000 queries per hour, set the query_times as 1000')


    temp_url = self.url.format(self.symbol)

    if not continue_last_run:
      if self.maxId != None and self.maxId != '':
        temp_url += 'max={}'.format(int(self.maxId)-1)
      self.df = pd.DataFrame()
    else:
      temp_url += 'max={}'.format(int(self.lastRun)-1)

    try:
      for i in range(query_times):
        response = requests.get(temp_url)
        messages = json.loads(response.content)['messages']
        lastid = messages[-1]['id']
        self.df = self.df.append(messages,ignore_index=True)
        temp_url = self.url.format(self.symbol) + 'max={}'.format(int(lastid)-1)
        if not silent:
          print('run query {} time'.format(i+1))
        time.sleep(3)
      print('finished, {} queries in total this time'.format(query_times))
    except Exception as e:
      self.lastRun = lastid
      self.lastStatus = response.status_code
      print(e)
      print('something wrong or user stopped')
    
    self.lastRun = lastid
    self.lastStatus = response.status_code




In [ ]:
app = StockTwitAPIScrapper()

In [ ]:
if not os.path.exists(app.symbol):
    os.makedirs(app.symbol)
flag = 1
while flag==1:
    try:
        app.scrap(silent=True)
        app.df.to_csv(app.symbol+'/'+app.symbol+'_'+str(app.df.id[0])+'_'+str(app.lastRun)+'.csv',encoding='utf-8-sig')
        app.df = pd.DataFrame()
    except Exception as e:
        print(e)
        flag=0
        break

In [ ]:
# check all the entry follow descending time sequence
for i in range(len(app.df.id)-1):
  if app.df.id[i] - app.df.id[i+1] < 0:
    print(i)
    print(app.df.id[i] - app.df.id[i+1])

#check last run id and last run status code
print(app.lastRun)
print(app.lastStatus)